In [89]:
!pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 2.7/2.7 GB 931.0 kB/s eta 0:00:00
     ---------------------------------------- 4.9/4.9 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 4.0/4.0 MB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 1.4 MB/s eta 0:00:00


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.11.0 requires keras<2.12,>=2.11.0, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.11.0 requires tensorboard<2.12,>=2.11, but you have tensorboard 2.10.1 which is incompatible.
tensorflow-intel 2.11.0 requires tensorflow-estimator<2.12,>=2.11.0, but you have tensorflow-estimator 2.10.0 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 536.2/536.2 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1
    Uninstalling torchaudio-0.12.1:
      Successfully uninstalled torchaudio-0.12.1


In [148]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-large", device=0)

#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
#model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large", device=0)

In [116]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")
#model = AutoModelForQuestionAnswering.from_pretrained("google/flan-t5-large")



Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
import pandas as pd

In [138]:
df =  pd.read_csv('data/li_jobs.zip')
df = pd.read_parquet('data/data_processed.parquet')

In [158]:
context = df.sample(100).job_description.iloc[8].replace('\\n', '\n')
question = 'What skills are required for this job?'

In [211]:
import torch

inputs = tokenizer(question, context, return_tensors="pt", max_length=512)

with torch.no_grad():
     outputs = model(**inputs)
    
    
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

answer_start_index = max(0, answer_start_index - 10)
answer_end_index = answer_end_index + 10

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

print(tokenizer.decode(predict_answer_tokens))

. quantzig. com / job description : technical skills : • expert knowledge in object - oriented python programming. • proficient in pushing code to git for


In [160]:
#outputs.start_logits.sort()

In [161]:
text = question + '\n' + context


pipe(text)

[{'generated_text': 'An opportunity has arisen for an experienced Head of Safety Operations, or similar, with a'}]

# Split job description and classify it (has skills or not)

In [162]:
dfs = df.sample(500)

In [138]:
dfs[['job_description']].to_excel('raw_desc.xlsx')

In [163]:
flattened = [x for corpus in dfs.job_description.to_list() for x in corpus.replace('\\n', '\n').split('\n\n') if x]

In [80]:
pd.Series(flattened).to_frame().to_excel('split_desc.xlsx')

In [164]:
context = flattened[106]
text = 'Q: Does it description of skills required for a job? \n' + context 

crop_flattened = [context for context in flattened if len(context)>50]

all_text = ['Context: ' + context + '\nQ: is this a description of skills required for a job?' for context crop_flattened]

In [165]:
ans = pipe(all_text)

In [176]:
sum([1 if a['generated_text'] == 'yes' else 0 for a in ans])

1690

In [177]:
len([1 if a['generated_text'] == 'yes' else 0 for a in ans])

4032

In [170]:
num=14

print(ans[num])
print(all_text[num])

{'generated_text': 'yes'}
Context: If you’re interested in this role and have many, but not all, of the experiences needed, we encourage you to apply. You may still be the right candidate for this or other opportunities at Philips. Learn more about our commitment to diversity and inclusion here.
Q: is this a description of skills required for a job?


In [133]:
all_text

['Context: Azienda leader nella distribuzione e nelle-commerce nel settore editoriale\nQ: is this a description of skills required for a job?',
 'Context: Analisi, sviluppo e implementazione degli applicativi proprietari sia interni che esterni per i clienti\nQ: is this a description of skills required for a job?',
 'Context: Diploma e/o Laurea in informatica o affine\nPregressa esperienza di almeno 2 anni nello sviluppo software\nOttima conoscenza del linguaggio C# e di .NET Framework\nCapacità di progettazione e scrittura di query avanzate per Microsoft SQL Server\nFamiliarità con servizi principali AWS come EC2, S3 e IAM\nFamiliarità con servizi principali Azure come VM, Blob Storage e Azure AD\nPlus: esperienza su sistemi ERP enterprise\nShow less\nQ: is this a description of skills required for a job?',
 'Context: Quality Engineer Stockport Up to £48,000 Are you passionate about ensuring excellence in manufacturing and improving processes? Do you thrive in environments where atten

In [134]:
flattened

['Azienda leader nella distribuzione e nelle-commerce nel settore editoriale',
 'Posizione:',
 'Analisi, sviluppo e implementazione degli applicativi proprietari sia interni che esterni per i clienti',
 'Requisiti:',
 'Diploma e/o Laurea in informatica o affine\nPregressa esperienza di almeno 2 anni nello sviluppo software\nOttima conoscenza del linguaggio C# e di .NET Framework\nCapacità di progettazione e scrittura di query avanzate per Microsoft SQL Server\nFamiliarità con servizi principali AWS come EC2, S3 e IAM\nFamiliarità con servizi principali Azure come VM, Blob Storage e Azure AD\nPlus: esperienza su sistemi ERP enterprise\nShow less',
 'Quality Engineer Stockport Up to £48,000 Are you passionate about ensuring excellence in manufacturing and improving processes? Do you thrive in environments where attention to detail and precision are paramount? Join my clients team, a renowned multinational engineering manufacturer, as a Quality Engineer. As a Quality Engineer, you will; E

In [185]:
splitted = pd.Series(crop_flattened).to_frame()
marks = [1 if a['generated_text'] == 'yes' else 0 for a in ans]

In [186]:
splitted['label'] = marks

In [187]:
splitted

,0,label
0,Job Title\nRegional Manager - Customer Service...,0
1,Your role:\nRCSM is responsible for high emplo...,1
2,Customer Support caters to all customers with ...,0
3,The diversity of the business runs from handli...,1
4,Critical care systems like anesthesia and vent...,0
...,...,...
4027,La saisie des opérations courantes ;\nLétablis...,1
4028,Vos missions pourront évoluer en fonction de v...,0
4029,Mutuelle dentreprise.\nCouverture prévoyance.\...,0
4030,"De formation Bac +2/3 minimum, idéalement ou d...",1


In [189]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

C:\Users\Vyacheslav\PycharmProjects\CV_projects\venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vyacheslav\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [195]:
inputs = tokenizer(splitted[0].to_list()[0], return_tensors="pt", max_length=512)

In [196]:
inputs

{'input_ids': tensor([[ 101, 3105, 2516, 3164, 3208, 1011, 8013, 2326, 1011, 2148,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [198]:
tok = splitted[0].map(lambda x: tokenizer(x, return_tensors="pt", max_length=512))

In [200]:
tok.iloc[6]

{'input_ids': tensor([[  101,  2057,  2024,  1037,  2740,  2974,  2194,  1012,  2057,  2328,
          2256,  2972,  2194,  2105,  1996,  6772,  2008,  2296,  2529,  5609,
          1010,  1998,  2057,  2180,  2102,  2644,  2127,  7955,  7249,  2038,
          3229,  2000,  1996,  3737,  9871,  2008,  2057,  2035, 10107,  1012,
          2079,  1996,  2147,  1997,  2115,  2166,  2000,  2393,  1996,  3268,
          1997,  2500,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}

## Text Generation

In [1]:
#!pip install accelerate
!pip install flash-attn --no-build-isolation

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [1847 lines of output]
  fatal: not a git repository (or any of the parent directories): .git
  
  
  torch.__version__  = 2.2.0+cu118
  
  
  C:\Users\Vyacheslav\PycharmProjects\CV_projects\venv\lib\site-packages\setuptools\__init__.py:80: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!
  
          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can try `pip install --use-pep517`.
          ********************************************************************************
  
  !!
    dist.fetch_build_eggs(dist.setup_requires)
  running bdist_wheel
  Guessing wheel URL:  https://github.com/Dao-AILab/flash-attention/releases/download/v2.5.8/flash_attn-2.5.8+cu118torch2.2cxx11abiFALSE-cp39-cp39-win_amd64


  Using cached flash_attn-2.5.8.tar.gz (2.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached einops-0.8.0-py3-none-any.whl.metadata (12 kB)
  Using cached ninja-1.11.1.1-py2.py3-none-win_amd64.whl.metadata (5.4 kB)
Using cached einops-0.8.0-py3-none-any.whl (43 kB)
Using cached ninja-1.11.1.1-py2.py3-none-win_amd64.whl (312 kB)
  Running setup.py clean for flash-attn
Failed to build flash-attn


In [2]:
#from transformers import pipeline

#pipe = pipeline("text-generation", model="microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True, device = 0, attn_implementation='eager')

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:

model_name = "microsoft/Phi-3-mini-128k-instruct"
model_name = "microsoft/Phi-3-mini-4k-instruct-fp16.gguf" 
model_name = "microsoft/Phi-3-mini-4k-instruct-gguf"

# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, 
                                             #attn_implementation='eager',
                                             #device_map = 'cuda'
                                            )

C:\Users\Vyacheslav\PycharmProjects\CV_projects\venv\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


OSError: microsoft/Phi-3-mini-4k-instruct-gguf does not appear to have a file named config.json. Checkout 'https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/main' for available files.

In [20]:
import pandas as pd
df =  pd.read_csv('data/li_jobs.zip')
df = pd.read_parquet('data/data_processed.parquet')

In [4]:
text = "list top 5 skills for project manager CV in short form"

instruct = f"Instruct: {text}\nOutput:"

In [5]:
token_ids = tokenizer.encode(instruct, add_special_tokens=False, return_tensors="pt")

In [6]:
with torch.no_grad():
    output_ids = model.generate(
      token_ids.to(model.device),
      do_sample = True,
      temperature = 0.7,
      max_new_tokens = 256
  )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)

print(output)

C:\Users\Vyacheslav\PycharmProjects\CV_projects\venv\lib\site-packages\transformers\generation\utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


KeyboardInterrupt: 

In [15]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
tokenizer_gpt = GPT2Tokenizer.from_pretrained('gpt2-medium')
model_gpt = GPT2LMHeadModel.from_pretrained('gpt2-medium')


In [47]:
#text = "Help me to describe experience of senior data analyst: "
encoded_input = tokenizer_gpt.encode(text, return_tensors="pt")
output = model_gpt.generate(encoded_input, 
                      do_sample = True,
                      temperature = 0.7,
                      max_new_tokens = 128
                       ) 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [48]:
generated_text = list(map(tokenizer_gpt.decode, output))[0]
print(generated_text)

Job description: About the Role: We are seeking skilled Electrical Technicians to join our team for various offshore flowline hook-up projects in the Aberdeen area. The successful candidates will play a crucial role in ensuring the safe and efficient electrical operations.
Key Responsibilities:
Conduct electrical installations, maintenullce, and repairs as per project requirements.
Troubleshoot and rectify electrical issues efficiently to minimize downtime.
Perform routine inspections and tests on electrical systems and equipment.
Adhere strictly to safety protocols and procedures to ensure a safe working environment.
Collaborate effectively with other team members to achieve project objectives.
Complete all assigned tasks within specified deadlines and quality standards.
Maintain accurate documentation of work activities, including reports and records.
Requirements:
Proven experience as an Electrical Technician in offshore environments.
Proficiency in electrical systems installation, 

# Question Answering

In [7]:
import pandas as pd
df =  pd.read_csv('data/li_jobs.zip')
df = pd.read_parquet('data/data_processed.parquet')

In [67]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True, 
#                                              attn_implementation='eager',
#                                              device_map = 'cuda'
#                                             )

In [68]:
context = df.sample(100).job_description.iloc[8].replace('\\n', '\n')
question = 'What skills are required for this job?'

text = 'Job description: ' + context + '\n' + question


In [72]:
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

In [73]:
with torch.no_grad():
    output_ids = model.generate(
      token_ids.to(model.device),
      do_sample = True,
      temperature = 0.7,
      max_new_tokens = 256
  )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)

print(output)



- explanation: The job description for a Skill Farm Member involves a variety of responsibilities that require a blend of technical, analytical, and interpersonal skills. Below is a detailed breakdown of the skills and qualifications required for this role:

1. **Educational Background**:
   - A Bachelor's degree in a STEM field is a prerequisite. This indicates that the role may involve aspects of science, technology, engineering, and mathematics.
   - At least a 2:1 (or an equivalent) degree level reflects a strong academic foundation.
   - A-levels with an A grade or higher in subjects related to the STEM field, particularly in Maths, suggest a high proficiency in mathematical concepts.

2. **Academic Skills**:
   - Strong mathematical skills, as the role requires handling invoices and other financial documents.
   - Proficiency in using Microsoft Office Suite (Word, Excel, PowerPoint) for creating invoices, reports, and other documents.

3. **Professional Qualifications**:
   - W

In [71]:
print(context)

Skill Farm Members are invited to apply for the following role:

Responsibilities:

Sales Ledger: preparing invoices based on input data from other teams

Purchase ledger: posting invoices and preparing payment runs

Resolving customer and supplier invoice queries

Assisting with preparing the month end accounts

Assisting with annual processes such as budgets and year-end audit

Assisting with general office and administrative duties as necessary

Skills & Qualifications:

You have graduated (or are graduating) with a STEM Bachelors degree, having achieved at least 2:1 (or an equivalent)

You enjoy Maths and have achieved an A or higher at A-levels (or an equivalent)

You are prepared to commit to a three year ACCA study programme, balancing full time work and study

You are confident using technology, such as MS Office and Confluence, and are keen to learn
Show less


# Summary

In [85]:
context = df.sample(100).job_description.iloc[8].replace('\\n', '\n')
question = 'Summarize the following job description.'

text = 'Instruct:' + question + '\n\n' + 'Job description: ' + context +  '\n\nOutput:'

In [86]:
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

In [87]:
with torch.no_grad():
    output_ids = model.generate(
      token_ids.to(model.device),
      do_sample = True,
      temperature = 0.7,
      max_new_tokens = 256
  )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)

print(output)

Gi Group SpA is seeking a Data Entry/Support Resource for a prestigious university's Grants Office. This position involves inserting data into internal systems and supporting project rendering activities related to national and international funding projects, managing MIUR projects for Italian professors, and handling project documentation and administrative records.

Key responsibilities include:
- Supporting the Grants Office team in various project-related activities.
- Handling documentation related to MIUR projects and assisting professors.
- Archiving documents and entering data using internal management systems.

Requirements for the role include:
- Possession of a Diploma or Laurea.
- English proficiency at B1-B2 level.
- Strong written and oral communication skills.
- Knowledge of office management software and a digital mindset.
- Flexibility, strong interpersonal skills, teamwork ability, and proactiveness.

The position offers:
- A fixed-term contract with the possibility o

In [88]:
print(text)

Instruct:Summarize the following job description.

Job description: Gi Group SpA, Agenzia per il Lavoro (Aut. Min. 26/11/04 Prot. n. 1101-SG) ricerca per importante realtà universitaria una risorsa in qualità di:

DATA ENTRY – SUPPORTO AI PROGETTI DI RENDICONTAZIONE

La risorsa verrà inserita all’interno dell’ufficio Grants Office nella direzione Academic Services. Si occuperà di supportare il team nelle seguenti attività:

Supporto alle attività di rendicontazione legate ai progetti finullziati da enti nazionali ed internazionali
gestione dei progetti del MIUR dei docenti italiani
redazione della documentazione amministrativa
archiviazione documentale e inserimento dati attraverso l’utilizzo di gestionali interni

Requisiti:

Diploma o Laurea
Inglese medio (B1-B2)
buone capacità comunicative (scritte e orali)
conoscenza del pacchetto office e approccio digital
Flessibilità, buone doti relazionali, capacità di lavoro in team, proattività.

Si offre

Un contratto a tempo determinato in 

In [6]:
from llama_cpp import Llama


llm = Llama(
  model_path="./Phi-3-mini-4k-instruct-q4.gguf",  # path to GGUF file
  n_ctx=4096,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8, # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=35, # The number of layers to offload to GPU, if you have GPU acceleration available. Set to 0 if no GPU acceleration is available on your system.
)

prompt = "How to explain Internet to a medieval knight?"

# Simple inference example
output = llm(
  f"<|user|>\n{prompt}<|end|>\n<|assistant|>",
  max_tokens=256,  # Generate up to 256 tokens
  stop=["<|end|>"], 
  echo=True,  # Whether to echo the prompt
)

print(output['choices'][0]['text'])

llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from ./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count_kv u32         

<|user|>
How to explain Internet to a medieval knight?<|end|>
<|assistant|> Good morrow, noble Knight of the Round Table. I shall endeavor to describe this fantastical concept known as the 'Internet' to you in words that may resonate with your understanding. Imagine if thou wilt, an ethereal realm where messages travel at incredible speeds across great distances, much like a swift steed racing through diverse lands.

In this realm called the Internet, knowledge and information are boundless, housed within vast libraries that never close their doors. These libraries are interconnected by magical portals known as 'devices,' such as scrolls imbued with enchanting power (computers) or even wondrous mirror-like reflectors (smartphones).

The Internet serves as a grand assembly hall, wherein individuals from far and wide gather to share their wisdom, stories, and experiences. They communicate through an intricate dance of symbols called 'texts,' crafting messages in the form of letters or im

In [30]:
context = df.sample(100).job_description.iloc[8].replace('\\n', '\n')
context = df[df.job_description.str.contains('pyspark')].sample(1).job_description.iloc[0].replace('\\n', '\n')



question = 'What skills are required for this job?'
question = 'What technical skills are required for this job?'

prompt = 'Job description: ' + context + '\n' + question


In [31]:
prompt

'Job description: Why work for us?\n\nA career at Janus Henderson is more than a job, it’s about investing in a brighter future together.\nOur Mission at Janus Henderson is to help clients define and achieve superior finullcial outcomes through differentiated insights, disciplined investments, and world-class service. We will do this by protecting and growing our core business, amplifying our strengths, and diversifying where we have the right.\n\nOur Values are key to driving our success, and are at the heart of everything we do:\n\nClients Come First - Always | Execution Supersedes Intention | Together We Win Diversity Improves Results | Truth Builds Trust\n\nIf our mission, values, and purpose align with your own, we would love to hear from you!\n\nThe Company\n\nWe are a leading independent global asset manager, dedicated to delivering the best outcomes for our clients through a highly diversified range of actively managed products. We are truly global, supporting our individual an

In [32]:
# Simple inference example
output = llm(
  f"<|user|>\n{prompt}<|end|>\n<|assistant|>",
  max_tokens=1024,  # Generate up to 256 tokens
  stop=["<|end|>"], 
  echo=True,  # Whether to echo the prompt
)

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     860.25 ms
llama_print_timings:      sample time =      52.91 ms /   277 runs   (    0.19 ms per token,  5235.11 tokens per second)
llama_print_timings: prompt eval time =     640.58 ms /    11 tokens (   58.23 ms per token,    17.17 tokens per second)
llama_print_timings:        eval time =   33504.83 ms /   276 runs   (  121.39 ms per token,     8.24 tokens per second)
llama_print_timings:       total time =   34858.78 ms /   287 tokens


<|user|>
Job description: Why work for us?

A career at Janus Henderson is more than a job, it’s about investing in a brighter future together.
Our Mission at Janus Henderson is to help clients define and achieve superior finullcial outcomes through differentiated insights, disciplined investments, and world-class service. We will do this by protecting and growing our core business, amplifying our strengths, and diversifying where we have the right.

Our Values are key to driving our success, and are at the heart of everything we do:

Clients Come First - Always | Execution Supersedes Intention | Together We Win Diversity Improves Results | Truth Builds Trust

If our mission, values, and purpose align with your own, we would love to hear from you!

The Company

We are a leading independent global asset manager, dedicated to delivering the best outcomes for our clients through a highly diversified range of actively managed products. We are truly global, supporting our individual and ins

In [33]:
question = 'Write a summary about this job'

prompt = 'Job description: ' + context + '\n' + question

In [ ]:
# Simple inference example
output = llm(
  f"<|user|>\n{prompt}<|end|>\n<|assistant|>",
  max_tokens=1024,  # Generate up to 256 tokens
  stop=["<|end|>"], 
  echo=True,  # Whether to echo the prompt
)

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit
